## Model training and registration

In this notebook we will:

- Register a model to the model registry.
- Fetch the model from the model registry.

This will introduce the `hsml` (**H**opsworks **M**achine **L**earning) library, which contains functionality to keep track of models and deploy them.

*Note: you need to enable Serving to run this notebook. Go to the **old** Hopsworks UI, click on settings, and check the Serving box.*

In [1]:
import hsfs

conn = hsfs.connection()
fs = conn.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.


### Load Best Hyperparameters

We will train a model using the best hyperparameters we found in the previous notebook.
Recall that we saved these as a pickled dictionary that we uploaded to our cluster.
If you don't have the file locally you can download it using the *hopsworks* library.

In [2]:
import hopsworks

hopsworks_conn = hopsworks.connection()
project = hopsworks_conn.get_project()
dataset_api = project.get_dataset_api()

uploaded_file_path = "Resources/best_params.pickle"
dataset_api.download(uploaded_file_path)

Connected. Call `.close()` to terminate connection gracefully.


'/srv/hops/staging/private_dirs/73268cbe6c091b28dfed07bbcd22e4093fb6a96467f44093772636a10323e841/best_params.pickle'

Retrieve the best hyperparameters from this file.

In [3]:
import pickle

with open("best_params.pickle", "rb") as f:
    best_params = pickle.load(f)

pos_class_weight = best_params["pos_class_weight"]

print(pos_class_weight)

0.7048771928503996


### Load Training Data & Train Model

Next, we'll train a model in the same way as the previous notebook.

# TODO (Davit): label returns empty list


In [6]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder

# Load data.
td = fs.get_training_dataset("transactions_view_1",1)
X_train = td.read("train")
X_val = td.read("validation")

# One-hot encode categorical feature "category".
enc = OneHotEncoder(sparse=False)
one_hot_train = pd.DataFrame(enc.fit_transform(X_train[["category"]]))
one_hot_val = pd.DataFrame(enc.transform(X_val[["category"]]))
X_train = pd.concat([X_train.drop(columns="category"), one_hot_train], axis=1)
X_val = pd.concat([X_val.drop(columns="category"), one_hot_val], axis=1)

# Separate target feature from input features.
target = "fraud_label" #td.label[0]   
y_train = X_train.pop(target)
y_val = X_val.pop(target)

# Train model.
clf = LogisticRegression(class_weight={0: 1.0 - pos_class_weight, 1: pos_class_weight}, solver='liblinear')
clf.fit(X_train, y_train)

LogisticRegression(C=1.0,
                   class_weight={0: 0.2951228071496004, 1: 0.7048771928503996},
                   dual=False, fit_intercept=True, intercept_scaling=1,
                   l1_ratio=None, max_iter=100, multi_class='auto', n_jobs=None,
                   penalty='l2', random_state=None, solver='liblinear',
                   tol=0.0001, verbose=0, warm_start=False)

We will also compute some evaluation metrics, which we will register together with the model.

In [7]:
from sklearn.metrics import precision_recall_fscore_support, classification_report

preds = clf.predict(X_val)

precision, recall, fscore, _ = precision_recall_fscore_support(y_val, preds, average="binary")

metrics = {
    'precision': precision,
    'recall': recall,
    'fscore': fscore
}

print(classification_report(y_val, preds))


              precision    recall  f1-score   support

           0       1.00      1.00      1.00     21014
           1       0.88      0.19      0.32        36

    accuracy                           1.00     21050
   macro avg       0.94      0.60      0.66     21050
weighted avg       1.00      1.00      1.00     21050



### Register model

One of the features in Hopsworks is the model registry. This is where we can store different versions of models and compare their performance. Models from the registry can then be served as API endpoints.

Let's connect to the model registry using the [HSML library](https://docs.hopsworks.ai/machine-learning-api/latest) from Hopsworks.

# TODO (Davit): make sure serving is enabled in the settings


In [9]:
import hsml

conn = hsml.connection()
mr = conn.get_model_registry()

Connected. Call `.close()` to terminate connection gracefully.


Before registering the model we will export it as a pickle file using joblib.

In [10]:
import joblib

joblib.dump(clf, 'model.pkl')

['model.pkl']

The model needs to be set up with a [Model Schema](https://docs.hopsworks.ai/machine-learning-api/latest/generated/model_schema/), which describes the inputs and outputs for a model.

A Model Schema can be automatically generated from training examples, as shown below.

In [11]:
from hsml.schema import Schema
from hsml.model_schema import ModelSchema

input_schema = Schema(X_train)
output_schema = Schema(y_train)
model_schema = ModelSchema(input_schema=input_schema, output_schema=output_schema)

model_schema.to_dict()

{'input_schema': {'columnar_schema': [{'name': 'amount', 'type': 'float64'},
   {'name': 'age_at_transaction', 'type': 'float64'},
   {'name': 'days_until_card_expires', 'type': 'float64'},
   {'name': 'loc_delta', 'type': 'float64'},
   {'name': 'trans_volume_mavg', 'type': 'float64'},
   {'name': 'trans_volume_mstd', 'type': 'float64'},
   {'name': 'trans_freq', 'type': 'float64'},
   {'name': 'loc_delta_mavg', 'type': 'float64'},
   {'name': '0', 'type': 'float64'},
   {'name': '1', 'type': 'float64'},
   {'name': '2', 'type': 'float64'},
   {'name': '3', 'type': 'float64'},
   {'name': '4', 'type': 'float64'},
   {'name': '5', 'type': 'float64'},
   {'name': '6', 'type': 'float64'},
   {'name': '7', 'type': 'float64'},
   {'name': '8', 'type': 'float64'},
   {'name': '9', 'type': 'float64'}]},
 'output_schema': {'columnar_schema': [{'name': 'fraud_label',
    'type': 'int64'}]}}

With the schema in place, we can finally register our model.

In [12]:
model = mr.sklearn.create_model(
    name="fraud_tutorial_model",
    metrics=metrics,
    description="Logistic regression model trained with class weights.",
    input_example=X_train.sample(),
    model_schema=model_schema
)

model.save('model.pkl')


Exported model fraud_tutorial_model with version 1


Model('fraud_tutorial_model', 1, 'SKLEARN')

Here we have also saved an input example from the training data, which can be helpful for test purposes.

It's important to know that every time you save a model with the same name, a new version of the model will be saved, so nothing will be overwritten. In this way, you can compare several versions of the same model - or create a model with a new name, if you prefer that.

#### Finding the best performing model

Let's imagine you have trained and registered several versions of the same model. Now you can query the model registry for the best model according to your preferred criterion, say F1-score in our case.

The `direction` option is used to indicate if the metric should be high or low (max or min); in our case, it should be high (max).

In [13]:
best_model = mr.get_best_model(name="fraud_tutorial_model", metric="fscore", direction="max")
best_model.to_dict()

{'id': 'fraud_tutorial_model_1',
 'experimentId': None,
 'projectName': 'fraud_demo',
 'experimentProjectName': 'fraud_demo',
 'name': 'fraud_tutorial_model',
 'modelSchema': {'href': 'https://hopsworks.glassfish.service.consul:8182/hopsworks-api/api/project/119/dataset/Projects/fraud_demo/Models/fraud_tutorial_model/1/model_schema.json',
  'zip_state': 'NONE'},
 'version': 1,
 'description': 'Logistic regression model trained with class weights.',
 'inputExample': {'href': 'https://hopsworks.glassfish.service.consul:8182/hopsworks-api/api/project/119/dataset/Projects/fraud_demo/Models/fraud_tutorial_model/1/input_example.json',
  'zip_state': 'NONE'},
 'framework': 'SKLEARN',
 'metrics': {'precision': '0.875',
  'recall': '0.19444444444444445',
  'fscore': '0.3181818181818182'},
 'trainingDataset': None,
 'environment': ['/Projects/fraud_demo/Models/fraud_tutorial_model/1/environment.yml'],
 'program': 'Models/fraud_tutorial_model/1/program.ipynb'}

### Next Steps

In the next notebook, we'll look at model serving for the model we just registered to the Model Registry.